# Fazendo Recomendações #

<img src='RECOMENDO.jpg' >

Um algoritmo de recomendação basicamente usa os dados de preferências de um grupo de pessoas para fazer recomendações para outras pessoas. Um exemplo é a NetFlix que usa o seu histórico de filmes, séries e documentários assistidos para aprender suas preferêcias e encontrar o que pessoas parecidas com você assistiram e então te recomendar. 

### Coletando preferências ###

<img src='notas.png'>

A primeira coisa a se fazer é coletar dados das preferências de diferentes pessoas. Como exemplo, vamos criar alguns dados hipotéticos de avaliação de filmes. Uma ótima maneira de representar esses dados em Python e usar dicionários aninhados como se segue:

In [1]:
# Um dicionário de avaliações de filmes e suas classificações

criticas = {'Lisa Rose': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.5,
                         'Just My Luck': 3.0, 'Superman Returns': 3.5,
                         'You, Me and Dupree': 2.5,'The Night Listener': 3.0},
            
           'Gene Seymour': {'Lady in the Water': 3.0, 'Snakes on a Plane': 3.5,
                           'Just My Luck': 1.5, 'Superman Returns': 5.0,
                           'The Night Listener': 3.0, 'You, Me and Dupree': 3.5},
            
           'Michael Phillips': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.0,
                               'Superman Returns': 3.5, 'The Night Listener': 4.0},
            
           'Claudia Puig': {'Snakes on a Plane': 3.5, 'Just My Luck': 3.0,
                            'The Night Listener': 4.5, 'Superman Returns': 4.0,
                            'You, Me and Dupree': 2.5},
            
            'Mick LaSalle': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0,
                            'Just My Luck': 2.0, 'Superman Returns': 3.0,
                            'The Night Listener': 3.0, 'You, Me and Dupree': 2.0},
            
            'Jack Matthews': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0,
                             'The Night Listener': 3.0, 'Superman Returns': 5.0,
                             'You, Me and Dupree': 3.5},
            
            'Toby': {'Snakes on a Plane': 4.5, 'You, Me and Dupree': 1.0, 'Superman Returns': 4.0}
           }

#Obs: esse exemplo foi retirado do livro 'Programando a Inteligência Coletiva' de Toby Segaran, O'Reilly

### Encontrando Usuários Similares ###

Agora que temos esses dados, vamos descobrir o quão essas pessoas são parecidas criando alguma 'escala de similaridade' entre cada indivíduo. Podemos fazer isso de várias formas, duas delas seriam: distância euclidiana e correlação de pearson.  

* #### Usando a Distância Eucliadiana como escala de similaridade #### 

É algo bem simples, só usar os itens em comum que as pessoas categorizaram e os utilizar como eixos de um gráfico. Quanto mais próximas as pessoas estiverem no espaço de preferência, mais similares elas são. A seguir temos algumas pessoas plotadas num espaço de preferências, tendo como eixo os filmes 'You, Me and Dupree', e 'Snakes on a Plane'.

<img src='espaco_preferencia.png'> 

Vamos criar uma escala baseada na Distância Euclidiana. A seguir, temos uma função que nos dá uma espécie de nota de similaridade entre duas pessoas:

In [2]:
from math import *

def similaridade(dados_preferecias, pessoa1, pessoa2):
    '''
    Retorna uma nota de similaridade baseada na distância 
    euclidiana entre a pessoa1 e pessoa2.
    '''
    # obtem a lista de itens_em_comum entre a pessoa1 e pessoa2
    em_comum = {}
    for item in dados_preferecias[pessoa1]:
        if item in dados_preferecias[pessoa2]:
            em_comum[item] = 1
            
    # se não existe classificações em comum, retorna zero
    if len(em_comum) == 0: return 0
    
    # soma os quadrados de todas as diferenças entre as 
    # notas que pessoa1 e pessoa2 deu para um filme em comum
    soma_dos_quadrados = sum([pow(dados_preferecias[pessoa1][item] - dados_preferecias[pessoa2][item], 2) \
                             for item in dados_preferecias[pessoa1] if item in dados_preferecias[pessoa2]])
    
    # Calculando a escala dessa teremos que, quanto maior o valor mais semelhante 
    # as pessoas 1 e 2 são. Se usássemos apenas a distância euclidiana seria  
    # o contrário, quanto menor o valor, ou seja, a distância, mais semelhantes
    # elas seriam.  
    return 1 / (1 + soma_dos_quadrados)

In [3]:
# Vamos ver a nota de similaridade entre Lisa Rose e Gene Seymor
similaridade(criticas, 'Lisa Rose', 'Gene Seymour')

0.14814814814814814

* #### Usando o Coeficiente de correlação de Pearson como escala de similaridade ####

Uma forma um pouco mais sofisticada de determinar a similaridade entre duas pessoas seria usar o Coeficiente de Correlação de Pearson. Essa fórmula nos diz o quão bem os dados se encaixam em torno de uma reta, ou seja, o quão correlacionado esse conjunto de dados estão. Para uma explicação intuitiva sobre o Coeficiente de Correlação de Pearson, assista:

[The Correlation Coefficient - Explained in Three Steps](https://www.youtube.com/watch?v=ugd4k3dC_8Y)

A figura a seguir nos ajuda a visualizar esse método. Os eixos dessa vez são as pessoas, na figura temos como eixos Mick LaSalle e Gene Seymour, e os pontos no gráfico são os filmes em comum plotados de acordo com a nota que cada um dos dois deu. Por exemplo, Mick deu nota 3 para o filme Superman e Gene deu nota 5 para o mesmo, de modo que Supermen se encontra na coordenada (3, 5). 

<img src='grafico_dispersao.png'>

A reta que está encaixada entre os filmes é chamada de *melhor reta*, ou ainda, *linha mais adequada* para descrever esses dados. Essa reta é encontrada através de métodos de regresão linear(Veja o notebook *Usando Gradiente Descendente para fazer Regresão Linear*). 

A seguir, temos a fórmula do Coeficiente de Pearson: 

<img src='formulapearson.jpg'>

Não se assuste com essa fórmula, ela é muito mais símples do que parece. Caso deseje entender melhor o que tudo isso significa, assista esse exelente vídeo:

[Estatística - Aula 15: Coeficiente de Correlação](https://www.youtube.com/watch?v=RPOpNR387yg)

A seguir, temos a função que faz esse cálculo:

In [4]:
def coeficiente_pearson(dados_preferencias, pessoa1, pessoa2):
    '''
    Retorna o coeficiente de pearson entre pessoa1 e pessoa2
    '''
    
    # Obtem o dicionário de filmes em comum
    em_comum = {}
    for item in dados_preferencias[pessoa1]:
        if item in dados_preferencias[pessoa2]:
            em_comum[item] = 1
            
    # Se não tem filmes em comum, retorna zero
    n = len(em_comum)
    if n == 0:
        return 0
    
    # Soma todas as preferências
    soma1 = sum(dados_preferencias[pessoa1][filme] for filme in em_comum)
    soma2 = sum(dados_preferencias[pessoa2][filme] for filme in em_comum)
    
    # Soma os quadrados
    soma1_quadrado = sum([pow(dados_preferencias[pessoa1][filme], 2) for filme in em_comum])
    soma2_quadrado = sum([pow(dados_preferencias[pessoa2][filme], 2) for filme in em_comum])
    
    # Soma os produtos
    soma_produto = sum(dados_preferencias[pessoa1][filme]*dados_preferencias[pessoa2][filme] \
                       for filme in em_comum)
    
    # Calcula o coeficiente de pearson
    
    # (n(Σxy) - (Σx)(Σy))
    numerador = soma_produto - (soma1*soma2/n)
    # √((n(Σx²) - (Σx)²)(n(Σy²) - (Σy)²))
    denominador = sqrt((soma1_quadrado-pow(soma1, 2)/n)*(soma2_quadrado-pow(soma2,2)/n))

    if denominador == 0: return 0
    
    # r = (n(Σxy) - (Σx)(Σy)) ÷ √((n(Σx²) - (Σx)²)(n(Σy²) - (Σy)²))
    r = numerador/denominador
    
    return r
    

In [5]:
# Vamos ver o coeficiente de correlação entre Lisa e Gene

coeficiente_pearson(criticas, 'Lisa Rose', 'Gene Seymour')

0.39605901719066977

### Qual métrica de similaridade deve ser usada? ###

Existem inúmeras outras formas de calcular similaridade entre dois conjuntos de dados, tudo dependerá do seu conjunto de dados. Para conhecer outras métricas de similaridade leia:

link

### Classificando as avaliações ###

Agora que temos duas formas de calcular similaridades, poderemos procurar para cada pessoa as pessoas que mais se parecem com elas. A função seguinte cria uma lista ordenada de pessoas com gostos similares a determinada pessoa.

In [6]:
def mais_parecidas(dados_preferencias, pessoa, n=5, metrica=coeficiente_pearson):
    '''
    Retorna os melhores resultados para cada pessoa do dicioário de preferências. 
    O numero 'n', ou seja, o número de resultados e a métrica são parâmetros opcionais.
    '''
    # cria uma lista de tuplas (nota, qual_pessoa)
    notas = [(metrica(criticas, pessoa, outra), outra) \
            for outra in criticas if outra != pessoa]
    
    # Ordena a lista de notas na ordem decrescente
    notas.sort()
    notas.reverse()
    
    return notas[0:n]

In [7]:
# Vamos calcular as três pessoas mais parecidas com Toby
mais_parecidas(criticas, 'Toby', n=3)

[(0.9912407071619299, 'Lisa Rose'),
 (0.9244734516419049, 'Mick LaSalle'),
 (0.8934051474415647, 'Claudia Puig')]

### Recomendando itens ###

O que queremos fazer de verdade aqui é recomendar filmes. Uma forma bem intuitiva de fazer isso é olhar para aquelas pessoas mais parecidas a mim e ver os filmes que essas outras pessoas assistiram que eu ainda não assisti. Mas esse tipo de abordagem não seria tão eficaz. Como exemplo, essa abordagem poderia me retornar uma pessoa que não tenha feito uma avaliação de um filme que eu poderia ter gostado ou ainda, que essa pessoa tenha gostado de um filme em que todos os outros avaliaram mal.

Para contornar esses problemas, poderíamos dar notas para os itens usando a média ponderada que sirva para classificar avaliações. Pegue os votos de todos aqueles que fizeram avaliações para um determinado filme em comum e multiplique o quanto  eles são semelhantes a alguém pela nota que eles deram para cada item. 

In [11]:
def recomendacoes(dados_preferencia, pessoa, metrica=coeficiente_pearson):
    '''
    Obtem recomendações para uma pessoa usando
    uma média ponderada de cada avaliação dos 
    outros usuários. 
    '''
    
    totais = {}
    soma_similaridade = {}
    for outra in dados_preferencia:
        # não deixa que compare comigo mesmo
        if outra == pessoa: continue
        similaridade = metrica(dados_preferencia, pessoa, outra)
        
        # ignora notas zero ou valores menores que zero
        if similaridade <= 0: continue
            
        for filme in dados_preferencia[outra]:
            # apenas notas de filmes que eu não tenha visto ainda
            if filme not in dados_preferencia[pessoa] or dados_preferencia[pessoa][filme] == 0:
                # similaridade * nota
                totais.setdefault(filme, 0)
                totais[filme] += dados_preferencia[outra][filme] * similaridade
                
                # soma de similaridade
                soma_similaridade.setdefault(filme, 0)
                soma_similaridade[filme] += similaridade
                
        # cria a lista normalizada
        rankings = [(total/float(soma_similaridade[filme]), filme) for filme, total in totais.items()]
        
        # retorna a lista ordenada de forma decrescente
        rankings.sort()
        rankings.reverse()
        
        return rankings

In [12]:
# Vamos ver os filmes recomendados para Toby

recomendacoes(criticas, 'Toby')

[(3.0, 'The Night Listener'), (3.0, 'Lady in the Water')]

In [13]:
recomendacoes(criticas, 'Michael Phillips')

[(3.5, 'You, Me and Dupree')]